### Data Pre-processing (detailed explanations are included in data-processing.ipynb)

In [1]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
import pandas as pd
from requests import get

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [4]:
body = soup.find('body')
#print(body.prettify())

In [5]:
body_content = body.find('div',class_='mw-parser-output').table.tbody
print(len(body_content))
data_list = []

580


In [6]:
for idx, tr in enumerate(body_content.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        tds0 = tds[0]
        tds1 = tds[1].find('a')
        tds2 = tds[2].find('a')
        if tds1 == None and tds2 == None:   
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 != None and tds2 == None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds1.text,
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 == None and tds2 != None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds2.text
            })
        else:
             data_list.append({
             'PostalCode': tds0.contents[0],
              'Borough': tds1.text,
              'Neighborhood':tds2.text
              })

In [7]:
df = pd.DataFrame(data_list,columns = ['PostalCode','Borough','Neighborhood'])

In [8]:
df = df[(True^df['Borough'].isin(['Not assigned']))]

In [9]:
for x in df['Neighborhood']:
    if x in ('Not assigned','Not assigned\n'):
        df.loc[df.Neighborhood == x,'Neighborhood']=df.loc[df.Neighborhood == x,'Borough']
    else:
        pass

In [10]:
df['Neighborhood']=df['Neighborhood'].str.replace('\n','')
#print(df.to_string()) #a way to display all rows of data

In [11]:
dataset = df.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()
dataset = dataset.join(df.set_index('PostalCode')['Borough'], on='PostalCode').drop_duplicates()  ##set PostalCode column as Index!
Torontodata = pd.DataFrame(dataset,columns = ['PostalCode','Borough','Neighborhood'])
Torontodata = Torontodata.sample(frac=1).reset_index(drop=True)

In [12]:
Torontodata.shape

(103, 3)

### Download geographical data from csv and attach to original data. (Geocoder package cannot be downloaded)

In [13]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data')

In [14]:
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
Torontodata = Torontodata.join(geodata.set_index('PostalCode'), on='PostalCode')
Torontodata

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
1,M3B,North York,Don Mills North,43.745906,-79.352188
2,M9P,Etobicoke,Westmount,43.696319,-79.532242
3,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
6,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
7,M2P,North York,York Mills West,43.752758,-79.400049
8,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564
